In [ ]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns; sns.set()
%matplotlib inline
# NLP
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction import text 
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
# modeling
import sklearn.metrics as metrics
from sklearn import metrics, utils
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
# metrics
from sklearn import metrics, model_selection, svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, plot_confusion_matrix, roc_curve, auc, classification_report
import pickle

In [ ]:
df = pd.read_csv('/content/DataSets 1 - DataSets 1.csv')

In [ ]:
df.head(2)

,text,label
0,ইনিই হচ্ছেন ভারতের প্রতিরক্ষামন্ত্রী মনোহর পা...,Geopolitical
1,রেন্ডিয়াকে পৃথীবির মানচিএ থেকে মুচে ফেলতে হবে,Geopolitical


In [ ]:
doc_train, doc_test = train_test_split(df, test_size=0.3, random_state=42)

In [ ]:
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

In [ ]:
 import nltk
 nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
tagged_train = doc_train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r.label]), axis=1)
tagged_test = doc_test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r.label]), axis=1)

In [ ]:
tagged_train.values[30]

TaggedDocument(words=['আরে', 'ভারত', 'না', 'থাকলে', 'এই', 'কাংলাদেশীরা', 'পাকিসতানী', 'চোদন', 'খেয়ে', 'শেষ', 'হয়ে', 'যেত'], tags=['Geopolitical'])

In [ ]:
# train a doc2vec model, using only training data
dbow_model = Doc2Vec(vector_size=100, 
                alpha=0.025, 
                min_count=5,
                dm=1, epochs=100)

In [ ]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

# building vocabulary 
dbow_model.build_vocab([x for x in tqdm(tagged_train.values)])

100%|██████████| 2392/2392 [00:00<00:00, 812304.69it/s]


In [ ]:
%%time
# this cell takes about 26 seconds to run
for epoch in range(30):
    dbow_model.train(utils.shuffle([x for x in tqdm(tagged_train.values)]), total_examples=len(tagged_train.values), epochs=1)
    dbow_model.alpha -= 0.002
    dbow_model.min_alpha = dbow_model.alpha

100%|██████████| 2392/2392 [00:00<00:00, 577027.39it/s]


CPU times: user 8.05 s, sys: 1.83 s, total: 9.87 s
Wall time: 8.32 s


In [ ]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [ ]:
# train-test split
y_train, X_train = vec_for_learning(dbow_model, tagged_train)
y_test, X_test = vec_for_learning(dbow_model, tagged_test)

In [ ]:
logreg = LogisticRegression(n_jobs=1, C=1e5)

In [ ]:
%%time
logreg.fit(X_train, y_train)

CPU times: user 406 ms, sys: 226 ms, total: 632 ms
Wall time: 335 ms


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(C=100000.0, n_jobs=1)

In [ ]:
logreg_y_preds = logreg.predict(X_test)

In [ ]:
logreg_precision = precision_score(y_test, logreg_y_preds,average='micro')
logreg_recall = recall_score(y_test, logreg_y_preds,average='micro')
logreg_f1_score = f1_score(y_test, logreg_y_preds,average='micro')
logreg_f1_weighted = f1_score(y_test, logreg_y_preds, average='micro')

In [ ]:
print('Precision: {:.4}'.format(logreg_precision))
print('Recall: {:.4}'.format(logreg_recall))
print('F1 Score: {:.4}'.format(logreg_f1_score))
print('Weighted F1 Score: {:.4}'.format(logreg_f1_weighted))

Precision: 0.5984
Recall: 0.5984
F1 Score: 0.5984
Weighted F1 Score: 0.5984


In [ ]:
# creating dictionary with all metrics
metric_dict = {}
metric_dict['Baseline Logisitic Regression'] = {'precision': logreg_precision, 'recall': logreg_recall, 'f1_score': logreg_f1_score}

In [ ]:
SVM_baseline = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto', class_weight='balanced')

In [ ]:
%%time
# this cell takes about 26 seconds to run
# fit the training dataset on the classifier
SVM_baseline.fit(X_train, y_train)

CPU times: user 579 ms, sys: 4.23 ms, total: 584 ms
Wall time: 602 ms


SVC(class_weight='balanced', gamma='auto', kernel='linear')

In [ ]:
# predict the labels on validation dataset
SVM_y_preds = SVM_baseline.predict(X_test)

In [ ]:
SVM_precision = precision_score(y_test, SVM_y_preds, average='micro')
SVM_recall = recall_score(y_test, SVM_y_preds, average='micro')
SVM_f1_score = f1_score(y_test, SVM_y_preds, average='micro')
SVM_f1_weighted = f1_score(y_test, SVM_y_preds, average='micro')

In [ ]:
# printing evaluation metrics up to 4th decimal place
print('Testing Metrics for SVM Baseline with Lemmatization Features')
print('Precision: {:.4}'.format(SVM_precision))
print('Recall: {:.4}'.format(SVM_recall))
print('F1 Score: {:.4}'.format(SVM_f1_score))
print('Weighted F1 Score: {:.4}'.format(SVM_f1_weighted))

Testing Metrics for SVM Baseline with Lemmatization Features
Precision: 0.576
Recall: 0.576
F1 Score: 0.576
Weighted F1 Score: 0.576


In [ ]:
metric_dict['Baseline SVM'] = {'precision': SVM_precision, 'recall': SVM_recall, 'f1_score': SVM_f1_score, 'weighted_f1': SVM_f1_weighted}

In [ ]:
pd.DataFrame.from_dict(metric_dict, orient='index')

,precision,recall,f1_score,weighted_f1
Baseline Logisitic Regression,0.598441,0.598441,0.598441,NaN
Baseline SVM,0.576023,0.576023,0.576023,0.576023


In [ ]:
target_names = ['class 0', 'class 1','class 2','class 3','class 4']
# logistic regression baseline
print(classification_report(y_test, logreg_y_preds, target_names=target_names))
# SVM baseline
print(classification_report(y_test, SVM_y_preds, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.49      0.20      0.29        93
     class 1       0.69      0.80      0.74       416
     class 2       0.46      0.56      0.51       195
     class 3       0.56      0.53      0.55       159
     class 4       0.60      0.40      0.48       163

    accuracy                           0.60      1026
   macro avg       0.56      0.50      0.51      1026
weighted avg       0.59      0.60      0.58      1026

              precision    recall  f1-score   support

     class 0       0.44      0.49      0.47        93
     class 1       0.75      0.64      0.70       416
     class 2       0.45      0.56      0.50       195
     class 3       0.51      0.54      0.52       159
     class 4       0.53      0.50      0.51       163

    accuracy                           0.58      1026
   macro avg       0.54      0.55      0.54      1026
weighted avg       0.59      0.58      0.58      1026



In [ ]:
scorers = {
            'f1_score': make_scorer(f1_score, average='micro'),
            'precision_score': make_scorer(precision_score, average='micro'),
            'recall_score': make_scorer(recall_score, average='micro'),
            'accuracy_score': make_scorer(accuracy_score)
          }

NameError: ignored

In [ ]:
print("Precision Score : ",precision_score(y_test, y_pred, 
                                           pos_label='positive',
                                           average='micro'))
print("Recall Score : ",recall_score(y_test, y_pred, 
                                           pos_label='positive',
                                           average='micro'))

NameError: ignored